In [2]:
from db import Team, Athlete, Meet
import requests as r
from sqlmodel import select, Session
from db.database import SessionLocal
import json

In [3]:
api_url = 'http://127.0.0.1:8000/'

In [4]:
def get_all_teams(session: Session):
    stmt = select(Team)
    return session().exec(stmt).all()

def insert_meets_and_athletes(params):
    q = r.get(api_url + 'team/getInfo', params = params)
    meets = q.json()['schedule']
    roster = q.json()['roster']
    for meet in meets:
        response = r.post(api_url + 'meet/addMeet',data=json.dumps(meet))
        if response.status_code != 200:
            print(meet['name'], response.text)
    for athlete in roster:
        response = r.post(api_url + 'athlete/addAthlete', data=json.dumps(athlete))
        if response.status_code != 200:
            print(athlete['anet_id'], response.text)
    return

In [19]:
season = 2018
sport = 'xc'
for team in get_all_teams(SessionLocal):
    params = dict(team_id = team.anet_id, season = season, sport = sport)
    insert_meets_and_athletes(params)

First Day of Practice {"detail":[{"loc":["body","venue"],"msg":"none is not an allowed value","type":"type_error.none.not_allowed"}]}
Saturday Practice at Lower Woodland Park {"detail":[{"loc":["body","venue"],"msg":"none is not an allowed value","type":"type_error.none.not_allowed"}]}
Saturday Practice at Lincoln Park {"detail":[{"loc":["body","venue"],"msg":"none is not an allowed value","type":"type_error.none.not_allowed"}]}
12241753 {"detail":"Athlete already exists"}
14082839 {"detail":"Athlete already exists"}
12317249 {"detail":"Athlete already exists"}
12379750 {"detail":"Athlete already exists"}
14082840 {"detail":"Athlete already exists"}
14109347 {"detail":"Athlete already exists"}
16738474 {"detail":"Athlete already exists"}
10509760 {"detail":"Athlete already exists"}
14188070 {"detail":"Athlete already exists"}
10509601 {"detail":"Athlete already exists"}
13123271 {"detail":"Athlete already exists"}
12317257 {"detail":"Athlete already exists"}
14082854 {"detail":"Athlete

In [8]:
from sqlalchemy import extract
def get_all_meets_by_season(session: Session, season: int):
    stmt = select(Meet).where(extract('year', Meet.date) == season)
    return session.exec(stmt).all()

In [20]:

meets = get_all_meets_by_season(SessionLocal(),2018)




In [21]:
def insert_new_result(meet_id: int):
    params = dict(sport = 'xc', meet_id = meet_id)
    s = r.get(api_url + 'meet/getResults',params=params)
    s.json()

    all_results = []
    for race in s.json()['races']:
        all_results += race['results']

    for result in all_results:
        response = r.post(api_url + 'meet/addResult', data = json.dumps(result))
        if response.status_code == 422:
            print(response.text)
    return "All done!"

In [22]:
for meet in meets:
    insert_new_result(meet.anet_id)

{"detail":[{"loc":["body","anet_athlete_id"],"msg":"none is not an allowed value","type":"type_error.none.not_allowed"}]}
{"detail":[{"loc":["body","anet_athlete_id"],"msg":"none is not an allowed value","type":"type_error.none.not_allowed"}]}
{"detail":[{"loc":["body","anet_athlete_id"],"msg":"none is not an allowed value","type":"type_error.none.not_allowed"}]}
{"detail":[{"loc":["body","anet_athlete_id"],"msg":"none is not an allowed value","type":"type_error.none.not_allowed"}]}
{"detail":[{"loc":["body","anet_athlete_id"],"msg":"none is not an allowed value","type":"type_error.none.not_allowed"}]}
{"detail":[{"loc":["body","anet_athlete_id"],"msg":"none is not an allowed value","type":"type_error.none.not_allowed"}]}
{"detail":[{"loc":["body","anet_athlete_id"],"msg":"none is not an allowed value","type":"type_error.none.not_allowed"}]}
{"detail":[{"loc":["body","anet_athlete_id"],"msg":"none is not an allowed value","type":"type_error.none.not_allowed"}]}
{"detail":[{"loc":["body